In [25]:
import pandas as pd
import numpy as np
import os
import pandas as pd

from fancyimpute import IterativeImputer
from functools import reduce

from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer

### OUTPUT: 'META' DATAFRAME FOR THE FOLLOWING STPES

## Merging patient visit and patient info
def create_var(patient_visit, patient_info):
    '''A function that takes in deid_cea_v2.csv and Final dataset prep_072521.csv to
    create all necessary variables. 
    Input those the two files and will return a combined file with all the new variables.
    '''
    ## Merging patient visit and patient meta
    df = patient_info.merge(patient_visit,on="PID", how='outer').sort_index()

    ## Creating all variables 
    # How long from the start of surveillance
    df['days_from_surveil'] = df['dx2cea'] - df['dx2surveildate'] 

    # How long from the previous visit
    df['days_from_last_visit'] = df['dx2cea'].diff() 

    # How long from the start of surveillance to their first visit
    df['first_visit_from_surveil'] = df.groupby('PID').head(1)['dx2cea'] - df.groupby('PID').head(1)['dx2surveildate']
    df['first_visit_from_surveil'] = df['first_visit_from_surveil'].ffill()

    # CEA value from previous visit 
    df['cea_prev_visit'] = df['value'].shift(1)
    
    # If CEA value >5, then higher chance of reoccurrence
    # First creating new value variable (value1) to convert all values into numeric to perform logic 
    df['value1'] = df['value'].astype(str)
    df['value1'] = df['value1'].str.replace('<', '').str.replace('>', '')
    df['value1'] = pd.to_numeric(df['value1'], errors = 'coerce')
    df['chances_of_recur'] = np.where(df['value1'] > 5, 1, 0)

    # Patient revisits every 90 days 
    df['return_visit'] = np.where(df.groupby('PID')['dx2cea'].diff() <= 90, 1, 0)
    return df


def combine_physid(directory, pattern):
    """A function that reads in all the physician characteristics files *md*.csv. 
    Then comibne all the physical characteristics files. 
    """
    # Use glob to find all files matching the pattern
    file_pattern = os.path.join(directory, pattern)
    csv_files = glob.glob(file_pattern)

    # Create an empty list to store individual DataFrames
    dataframes = []

    # Read each CSV file and append its contents to the list
    for csv_file in csv_files:
        df = pd.read_csv(csv_file, encoding='windows-1254')
        dataframes.append(df)

    # Concatenate all DataFrames in the list into a single DataFrame
    combined_data = reduce(lambda x, y: pd.merge(x, y, on = 'physid'), dataframes)
    return combined_data


In [26]:
import pandas as pd
import numpy as np
import os
import glob
from functools import reduce
import sys

sys.path.append("./data_processing/")
from create_variable import create_var, combine_physid

### Data Processing
# Read in data
patient_visit = pd.read_csv("./data/deid_cea_v2.csv") # Patient revisit after surveillance 
patient_info = pd.read_csv("./data/Final dataset prep_072521.csv") # All features data
patient = create_var(patient_visit, patient_info)

# Read in physician data and combine all physician data
directory_path = "./data/"
physid_pattern = "*md*.csv"
phys_meta = combine_physid(directory_path, physid_pattern)

# Combine patient_meta and physician_meta to create a meta file for downstream
patient_phys_info = patient.merge(phys_meta, how="left", left_on="physid_x", right_on="physid").drop(columns=['physid_x', 'physid_y']) # use "patient_phys_info" dataframe to continue working in dataframe
patient_phys_info.to_csv("./data/patient_phys_info.csv") # use "patient_phys_info.csv" to read in the information and continue working in another file

### Clean up Nans
### Model
### Model fitting

In [4]:
patient_phys_info.tail()

,Unnamed: 0,X.1,PID,X,srgprmsum,medctr2,age_deid,female_x,raceeth,partnered,...,employment_yr,termination_yr,female_y,ETHNICITY,JOB_TITLE,firstencyr,lastencyr,encyrs,PATCT,patperyr
569776,NaN,NaN,6822,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
569777,NaN,NaN,6822,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
569778,NaN,NaN,6822,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
569779,NaN,NaN,6822,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
569780,NaN,NaN,6822,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
#Exploratory Data Analysis
#Exploratory data analysis should describe the missingness
patient_phys_info.isnull().values.any()

True

In [6]:
#Number of missing values for each variable
mis_val_var_specific=(patient_phys_info.isnull().sum())

In [7]:
print('Number of missing values for each variable:')
pd.set_option('display.max_rows', None)  # Set to None to display all rows
print(mis_val_var_specific)

Number of missing values for each variable:
Unnamed: 0                   35463
X.1                          35463
PID                              0
X                            35463
srgprmsum                    35463
medctr2                      35463
age_deid                     35463
female_x                     35463
raceeth                      35463
partnered                    35463
lang                         35463
ins_medicare                 38169
ins_medicaid                 38169
ins_privatepay               38169
ins_commercial               38169
ins_other                    38169
charlson_wt                  37050
memb_1yrprior                35463
memb_at_dx                   35463
dx2membend                   38169
dx2membstart                 38169
dodi                         35463
dx2dod                      439705
IS_TOBACCO_USER              40508
race_new                     35463
HISPANIC                     35463
charlson_wt_nocancer         37050
rectalca   

In [9]:
#Unique number of patients
unique_patient_ids = patient_phys_info['patient_id'].nunique()

In [10]:
unique_patient_ids

6571

In [11]:
# Calculate the sum of the number of rows
num_rows_df = patient_phys_info.shape[0]

In [12]:
num_rows_df

569781

In [22]:
complete_cases_count = df.dropna().count()

print("Number of complete cases for each column:")
print(complete_cases_count)

Number of complete cases for each column:
Unnamed: 0                  0
X.1                         0
PID                         0
X                           0
srgprmsum                   0
medctr2                     0
age_deid                    0
female_x                    0
raceeth                     0
partnered                   0
lang                        0
ins_medicare                0
ins_medicaid                0
ins_privatepay              0
ins_commercial              0
ins_other                   0
charlson_wt                 0
memb_1yrprior               0
memb_at_dx                  0
dx2membend                  0
dx2membstart                0
dodi                        0
dx2dod                      0
IS_TOBACCO_USER             0
race_new                    0
HISPANIC                    0
charlson_wt_nocancer        0
rectalca                    0
icdo3site                   0
dxyr                        0
stage                       0
stage_char                  

Original DataFrame:


In [34]:
df.isnull().values.any()

True

In [42]:
# Columns to be removed
#'rt_sx', 'earlychemoname', 'earlychemorecur', and 'as.numeric.NA
columns_to_remove = ['rt_sx', 'earlychemoname', 'earlychemorecur','as.numeric.NA.']

# Drop specified columns
df_removed = df.drop(columns=columns_to_remove)

# Assuming your data is in a DataFrame named 'df'
# Replace 'your_dataset.csv' with the actual file path or URL to your dataset
# df = pd.read_csv('your_dataset.csv')

from sklearn.impute import SimpleImputer

# simple imputation using IterativeImputer
imp = SimpleImputer(strategy="most_frequent")
print('Number of missing values for each variable:')
pd.set_option('display.max_rows', None)  
print(imp.fit_transform(df_removed))

Number of missing values for each variable:
[[1.0 1.0 1 ... 17.0 3705.0 217.94117647]
 [1.0 1.0 1 ... 17.0 3705.0 217.94117647]
 [1.0 1.0 1 ... 17.0 3705.0 217.94117647]
 ...
 [3675.0 3675.0 6822 ... 17.0 6197.0 364.52941176]
 [3675.0 3675.0 6822 ... 17.0 6197.0 364.52941176]
 [3675.0 3675.0 6822 ... 17.0 6197.0 364.52941176]]


In [43]:
df_removed.head()

,Unnamed: 0,X.1,PID,X,srgprmsum,medctr2,age_deid,female_x,raceeth,partnered,...,employment_yr,termination_yr,female_y,ETHNICITY,JOB_TITLE,firstencyr,lastencyr,encyrs,PATCT,patperyr
0,1.0,1.0,1,1.0,30.0,9.0,73.0,Male,1.0,Others,...,2004.0,NaN,1.0,ASIAN,IM ONCOLOGY AND HEMATOLOGY,2004.0,2020.0,17.0,3705.0,217.941176
1,1.0,1.0,1,1.0,30.0,9.0,73.0,Male,1.0,Others,...,2004.0,NaN,1.0,ASIAN,IM ONCOLOGY AND HEMATOLOGY,2004.0,2020.0,17.0,3705.0,217.941176
2,1.0,1.0,1,1.0,30.0,9.0,73.0,Male,1.0,Others,...,2004.0,NaN,1.0,ASIAN,IM ONCOLOGY AND HEMATOLOGY,2004.0,2020.0,17.0,3705.0,217.941176
3,1.0,1.0,1,1.0,30.0,9.0,73.0,Male,1.0,Others,...,2004.0,NaN,1.0,ASIAN,IM ONCOLOGY AND HEMATOLOGY,2004.0,2020.0,17.0,3705.0,217.941176
4,1.0,1.0,1,1.0,30.0,9.0,73.0,Male,1.0,Others,...,2004.0,NaN,1.0,ASIAN,IM ONCOLOGY AND HEMATOLOGY,2004.0,2020.0,17.0,3705.0,217.941176


In [47]:
df_filled_mode = df_removed.fillna(df_removed.mode().iloc[0])  # Using iloc[0] to get the first mode if multiple modes exist

In [48]:
df_filled_mode.head()

,Unnamed: 0,X.1,PID,X,srgprmsum,medctr2,age_deid,female_x,raceeth,partnered,...,employment_yr,termination_yr,female_y,ETHNICITY,JOB_TITLE,firstencyr,lastencyr,encyrs,PATCT,patperyr
0,1.0,1.0,1,1.0,30.0,9.0,73.0,Male,1.0,Others,...,2004.0,2015.0,1.0,ASIAN,IM ONCOLOGY AND HEMATOLOGY,2004.0,2020.0,17.0,3705.0,217.941176
1,1.0,1.0,1,1.0,30.0,9.0,73.0,Male,1.0,Others,...,2004.0,2015.0,1.0,ASIAN,IM ONCOLOGY AND HEMATOLOGY,2004.0,2020.0,17.0,3705.0,217.941176
2,1.0,1.0,1,1.0,30.0,9.0,73.0,Male,1.0,Others,...,2004.0,2015.0,1.0,ASIAN,IM ONCOLOGY AND HEMATOLOGY,2004.0,2020.0,17.0,3705.0,217.941176
3,1.0,1.0,1,1.0,30.0,9.0,73.0,Male,1.0,Others,...,2004.0,2015.0,1.0,ASIAN,IM ONCOLOGY AND HEMATOLOGY,2004.0,2020.0,17.0,3705.0,217.941176
4,1.0,1.0,1,1.0,30.0,9.0,73.0,Male,1.0,Others,...,2004.0,2015.0,1.0,ASIAN,IM ONCOLOGY AND HEMATOLOGY,2004.0,2020.0,17.0,3705.0,217.941176


In [50]:
df_filled_mode.tail()

,Unnamed: 0,X.1,PID,X,srgprmsum,medctr2,age_deid,female_x,raceeth,partnered,...,employment_yr,termination_yr,female_y,ETHNICITY,JOB_TITLE,firstencyr,lastencyr,encyrs,PATCT,patperyr
569776,3675.0,3675.0,6822,3448.0,30.0,4.0,68.0,Male,1.0,"Married, common law, registered domestic partner",...,2004.0,2015.0,0.0,ASIAN,IM ONCOLOGY AND HEMATOLOGY,2004.0,2020.0,17.0,6197.0,364.529412
569777,3675.0,3675.0,6822,3448.0,30.0,4.0,68.0,Male,1.0,"Married, common law, registered domestic partner",...,2004.0,2015.0,0.0,ASIAN,IM ONCOLOGY AND HEMATOLOGY,2004.0,2020.0,17.0,6197.0,364.529412
569778,3675.0,3675.0,6822,3448.0,30.0,4.0,68.0,Male,1.0,"Married, common law, registered domestic partner",...,2004.0,2015.0,0.0,ASIAN,IM ONCOLOGY AND HEMATOLOGY,2004.0,2020.0,17.0,6197.0,364.529412
569779,3675.0,3675.0,6822,3448.0,30.0,4.0,68.0,Male,1.0,"Married, common law, registered domestic partner",...,2004.0,2015.0,0.0,ASIAN,IM ONCOLOGY AND HEMATOLOGY,2004.0,2020.0,17.0,6197.0,364.529412
569780,3675.0,3675.0,6822,3448.0,30.0,4.0,68.0,Male,1.0,"Married, common law, registered domestic partner",...,2004.0,2015.0,0.0,ASIAN,IM ONCOLOGY AND HEMATOLOGY,2004.0,2020.0,17.0,6197.0,364.529412


In [49]:
df_removed.isnull().values.any()

True

In [51]:
# Check for missing values in each column
missing_values = df_removed.isnull().sum()

# Print the variables with missing values
print("Variables with missing values:")
print(missing_values[missing_values > 0])

Variables with missing values:
Unnamed: 0               35463
X.1                      35463
X                        35463
srgprmsum                35463
medctr2                  35463
age_deid                 35463
female_x                 35463
raceeth                  35463
partnered                35463
lang                     35463
ins_medicare             38169
ins_medicaid             38169
ins_privatepay           38169
ins_commercial           38169
ins_other                38169
charlson_wt              37050
memb_1yrprior            35463
memb_at_dx               35463
dx2membend               38169
dx2membstart             38169
dodi                     35463
dx2dod                  439705
IS_TOBACCO_USER          40508
race_new                 35463
HISPANIC                 35463
charlson_wt_nocancer     37050
rectalca                 35463
icdo3site                35463
dxyr                     35463
stage                    35463
stage_char               35463
grade   